In [ ]:
from sqlalchemy import create_engine
from idetect.model import db_url, Base, Article, Session, Status

# connect to the DB specified in the docker.env file
engine = create_engine(db_url())
Session.configure(bind=engine)

# create the DB schema, if it doesn't already exist
Base.metadata.create_all(engine)

In [ ]:
session = Session()

In [ ]:
# what articles already exist?
[(a.url, a.status) for a in session.query(Article).order_by(Article.url).all()]

In [ ]:
# add an article to the database
article = Article(url="http://www.internal-displacement.org/", status=Status.NEW)
session.add(article)
session.commit()

In [ ]:
from idetect.worker import Worker

In [ ]:
# set up a worker that looks for NEW articles and runs a trivial function
worker = Worker(Status.NEW, Status.FETCHING, Status.FETCHED, Status.FETCHING_FAILED,
                lambda article: print(article.url), engine)

In [ ]:
# run that worker a single time
worker.work()

# note that if the flask/api container is running, there are some workers in there that will compete with this,
# so use `docker compose up notebooks` instead of just `docker compose up`